In [26]:
from pathlib import Path
import os
import json
import pandas as pd

folder_name = "results"
pardir = Path(__name__).parent
folder = os.path.join(pardir, folder_name)
models = [f for f in Path(folder).iterdir() if f.is_dir()]
contents = []
for model in models:
    files = [f for f in Path(model).iterdir() if f.is_file()]
    assert len(files) == 1, "should only have 1 result"
    file = files[0]
    with open(file, "r") as f:
        loaded = json.load(f)
        name = loaded['configs']['arc_challenge']['metadata']['pretrained']
        results = loaded['results']
        results['name'] = name
        contents.append(results)
df = pd.DataFrame(contents)
df=df.set_index("name")
df.head()

,arc_challenge,arc_easy,bbh,bbh_cot_fewshot_boolean_expressions,bbh_cot_fewshot_causal_judgement,bbh_cot_fewshot_date_understanding,bbh_cot_fewshot_disambiguation_qa,bbh_cot_fewshot_dyck_languages,bbh_cot_fewshot_formal_fallacies,bbh_cot_fewshot_geometric_shapes,...,mmlu_pro_economics,mmlu_pro_engineering,mmlu_pro_health,mmlu_pro_history,mmlu_pro_law,mmlu_pro_math,mmlu_pro_other,mmlu_pro_philosophy,mmlu_pro_physics,mmlu_pro_psychology
name,,,,,,,,,,,,,,,,,,,,,
allenai/Llama-3.1-Tulu-3-8B,"{'alias': 'arc_challenge', 'acc,none': 0.56569...","{'alias': 'arc_easy', 'acc,none': 0.8392255892...","{'exact_match,get-answer': 0.6891414529258179,...",{'alias': ' - bbh_cot_fewshot_boolean_expressi...,{'alias': ' - bbh_cot_fewshot_causal_judgement...,{'alias': ' - bbh_cot_fewshot_date_understandi...,{'alias': ' - bbh_cot_fewshot_disambiguation_q...,"{'alias': ' - bbh_cot_fewshot_dyck_languages',...",{'alias': ' - bbh_cot_fewshot_formal_fallacies...,{'alias': ' - bbh_cot_fewshot_geometric_shapes...,...,"{'alias': ' - economics', 'exact_match,custom-...","{'alias': ' - engineering', 'exact_match,custo...","{'alias': ' - health', 'exact_match,custom-ext...","{'alias': ' - history', 'exact_match,custom-ex...","{'alias': ' - law', 'exact_match,custom-extrac...","{'alias': ' - math', 'exact_match,custom-extra...","{'alias': ' - other', 'exact_match,custom-extr...","{'alias': ' - philosophy', 'exact_match,custom...","{'alias': ' - physics', 'exact_match,custom-ex...","{'alias': ' - psychology', 'exact_match,custom..."
allenai/OLMo-2-1124-7B-Instruct,"{'alias': 'arc_challenge', 'acc,none': 0.55119...","{'alias': 'arc_easy', 'acc,none': 0.8354377104...","{'exact_match,get-answer': 0.4904008600829366,...",{'alias': ' - bbh_cot_fewshot_boolean_expressi...,{'alias': ' - bbh_cot_fewshot_causal_judgement...,{'alias': ' - bbh_cot_fewshot_date_understandi...,{'alias': ' - bbh_cot_fewshot_disambiguation_q...,"{'alias': ' - bbh_cot_fewshot_dyck_languages',...",{'alias': ' - bbh_cot_fewshot_formal_fallacies...,{'alias': ' - bbh_cot_fewshot_geometric_shapes...,...,"{'alias': ' - economics', 'exact_match,custom-...","{'alias': ' - engineering', 'exact_match,custo...","{'alias': ' - health', 'exact_match,custom-ext...","{'alias': ' - history', 'exact_match,custom-ex...","{'alias': ' - law', 'exact_match,custom-extrac...","{'alias': ' - math', 'exact_match,custom-extra...","{'alias': ' - other', 'exact_match,custom-extr...","{'alias': ' - philosophy', 'exact_match,custom...","{'alias': ' - physics', 'exact_match,custom-ex...","{'alias': ' - psychology', 'exact_match,custom..."
marin-community/marin-8b-instruct,"{'alias': 'arc_challenge', 'acc,none': 0.53071...","{'alias': 'arc_easy', 'acc,none': 0.8152356902...","{'exact_match,get-answer': 0.26432191675625866...",{'alias': ' - bbh_cot_fewshot_boolean_expressi...,{'alias': ' - bbh_cot_fewshot_causal_judgement...,{'alias': ' - bbh_cot_fewshot_date_understandi...,{'alias': ' - bbh_cot_fewshot_disambiguation_q...,"{'alias': ' - bbh_cot_fewshot_dyck_languages',...",{'alias': ' - bbh_cot_fewshot_formal_fallacies...,{'alias': ' - bbh_cot_fewshot_geometric_shapes...,...,"{'alias': ' - economics', 'exact_match,custom-...","{'alias': ' - engineering', 'exact_match,custo...","{'alias': ' - health', 'exact_match,custom-ext...","{'alias': ' - history', 'exact_match,custom-ex...","{'alias': ' - law', 'exact_match,custom-extrac...","{'alias': ' - math', 'exact_match,custom-extra...","{'alias': ' - other', 'exact_match,custom-extr...","{'alias': ' - philosophy', 'exact_match,custom...","{'alias': ' - physics', 'exact_match,custom-ex...","{'alias': ' - psychology', 'exact_match,custom..."
mistralai/Mistral-Small-Instruct-2409,"{'alias': 'arc_challenge', 'acc,none': 0.58959...","{'alias': 'arc_easy', 'acc,none': 0.8472222222...","{'exact_match,get-answer': 0.7716172630932269,...",{'alias': ' - bbh_cot_fewshot_boolean_expressi...,{'alias': ' - bbh_cot_fewshot_causal_judgement...,{'alias': ' - bbh_cot_fewshot_date_understandi...,{

In [27]:
keys_to_try = ["acc,none", # default others
"prompt_level_strict_acc,none", #ifeval
"pass@1,create_test", #humaneval
"pass_at_1,none", #mbpp
"exact_match,custom-extract",
"exact_match,get-answer"] #default others
def extract(series): #expects you to do x['col']
    data = []
    for row in series:
        keys = list(row.keys())
        # import pdb; pdb.set_trace()
        for key in keys_to_try:
            if key in keys:
                data.append(row[key])
    return pd.Series(data, index = series.index)

df = df.apply(extract)

In [30]:
columns_we_care_abt = ['arc_challenge', 'arc_easy', 'bbh', 'ifeval', 'humaneval', 'mbpp']
df[columns_we_care_abt]

,arc_challenge,arc_easy,bbh,ifeval,humaneval,mbpp
name,,,,,,
allenai/Llama-3.1-Tulu-3-8B,0.565700,0.839226,0.689141,0.593346,0.628049,0.006
allenai/OLMo-2-1124-7B-Instruct,0.551195,0.835438,0.490401,0.591497,0.414634,0.278
marin-community/marin-8b-instruct,0.530717,0.815236,0.264322,0.497227,0.573171,0.024
mistralai/Mistral-Small-Instruct-2409,0.589590,0.847222,0.771617,0.574861,0.652439,0.570
ibm-granite/granite-3.2-8b-instruct,0.513652,0.801768,0.711411,0.628466,0.000000,0.540
